```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Ouput Layer](#output-layer)**


Originating story can be found here: [COR-681](https://logius-prd.jira.odc-noord.nl/browse/COR-681) and [COR-1056](https://logius-prd.jira.odc-noord.nl/browse/COR-1056)

# **FLOW DIAGRAMS**

---

`ADD DIAGRAM HERE!`

Required steps:

1. `ADD REQUIRMENT STEP LIST HERE!`

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>OL</font>**: Output Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Statics/pl_ProcessVaccinationCampaignMasterData.ipynb",
        "src/DataFactory/Protocols/pl_ProcessVaccinationPlannedNl.ipynb",
        "src/statics/masterdata_campaigns.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_VACCINATION_SERIES_NL]'))
CREATE TABLE [VWSSTAGE].[RIVM_VACCINATION_SERIES_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [VERSION] VARCHAR(50) NULL,
	[DATE_OF_REPORT] VARCHAR(50) NULL,
	[DATE_OF_STATISTICS] VARCHAR(50) NULL,
	[VACCINE_SERIE] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] VARCHAR(255) NULL
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[RIVM_VACCINATION_SERIES_NL]'))
CREATE TABLE [VWSINTER].[RIVM_VACCINATION_SERIES_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [VERSION] INT NULL,
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[VACCINE_SERIE] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] INT NULL	-- ADD COLUMNS
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO INTERMEDIATE.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_IL_RIVM_VACCINATION_SERIES_NL]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_VACCINATION_SERIES_NL] (
        [VERSION],
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS],
        [VACCINE_SERIE],
        [TOTAL_VACCINATED]
    )
    SELECT
        CAST([VERSION] as INT) as [VERSION],
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_REPORT]),
        [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_OF_STATISTICS]),
        UPPER(TRIM([VACCINE_SERIE])),
        CAST([TOTAL_VACCINATED] as INT) as [TOTAL_VACCINATED]
    FROM
        [VWSSTAGE].[RIVM_VACCINATION_SERIES_NL]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINATION_SERIES_NL])
END;
GO

# **OUTPUT LAYER**

---

## **<span style='color:teal'>RIVM VACCINATION CAMPAIGNS NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSDEST].[RIVM_VACCINATION_SERIES_NL]'))
CREATE TABLE [VWSDEST].[RIVM_VACCINATION_SERIES_NL] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[DATE_OF_REPORT] DATETIME NULL,
	[DATE_OF_STATISTICS] DATETIME NULL,
	[VACCINE_SERIE_EN] VARCHAR(255) NULL,
	[TOTAL_VACCINATED] INT NULL,
	[TOTAL_VACCINATED_7DAYS] INT NULL,
	[CUMSUM_TOTAL_VACCINATED] INT NULL  -- leave this column for later analysis
);
GO

-- 2) CREATE INDEX(S).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_VACCINATION_SERIES_NL] 
	ON [VWSDEST].[RIVM_VACCINATION_SERIES_NL]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_RIVM_VACCINATION_SERIES_NL]
    ON [VWSDEST].[RIVM_VACCINATION_SERIES_NL] (
		[DATE_LAST_INSERTED]
	)
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): INTERMEDIATE TO DESTINATION.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_OL_RIVM_VACCINATION_SERIES_NL]
AS
BEGIN
WITH CTE AS (
        SELECT 
			[ID],
            [DATE_OF_REPORT],
			[DATE_OF_STATISTICS],
            [VACCINE_SERIE],
            [TOTAL_VACCINATED]
        FROM [VWSINTER].[RIVM_VACCINATION_SERIES_NL]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_VACCINATION_SERIES_NL])
    ),
	CTE_7DAYS AS (
		SELECT 
			r.Id, 
			v.total_vaccinated
        FROM CTE r
		LEFT JOIN CTE v 
			ON r.VACCINE_SERIE = v.VACCINE_SERIE
			AND v.DATE_OF_STATISTICS > dateadd(day, -7,  [dbo].[WEEK_START_ISO](r.[DATE_OF_STATISTICS])) --Week before most recent sunday
			AND v.DATE_OF_STATISTICS <= [dbo].[WEEK_START_ISO](r.[DATE_OF_STATISTICS]) --Most recent sunday
	),
	CTE_7DAYS_SUM AS (
		SELECT Id AS Id, sum(total_vaccinated) AS SUM_VACCINATED
		FROM CTE_7DAYS
		group by Id
	),
	CTE_ALLDAYS AS (
		SELECT 
			r.Id, 
			v.total_vaccinated
        FROM CTE r
		LEFT JOIN CTE v 
			ON r.VACCINE_SERIE = v.VACCINE_SERIE
			AND v.DATE_OF_STATISTICS <= [dbo].[WEEK_START_ISO](r.[DATE_OF_STATISTICS]) --Most recent sunday
	),
	CTE_ALLDAYS_SUM AS (
		SELECT Id AS Id, sum(total_vaccinated) AS SUM_VACCINATED
		FROM CTE_ALLDAYS
		group by Id
	)
    INSERT INTO [VWSDEST].[RIVM_VACCINATION_SERIES_NL] (
            [DATE_OF_REPORT],
			[DATE_OF_STATISTICS],
            [VACCINE_SERIE_EN],
            [TOTAL_VACCINATED],
            [TOTAL_VACCINATED_7DAYS],
            [CUMSUM_TOTAL_VACCINATED]
    )
    SELECT
        c.[DATE_OF_REPORT],
        c.[DATE_OF_STATISTICS],
		c.[VACCINE_SERIE] As VACCINE_SERIE_EN,
        c.[TOTAL_VACCINATED],
		c7.SUM_VACCINATED AS TOTAL_VACCINATED_7DAYS,
		ca.SUM_VACCINATED AS CUMSUM_TOTAL_VACCINATED
    FROM CTE c
	INNER JOIN CTE_7DAYS_SUM c7 ON c.ID = c7.Id
	INNER JOIN CTE_ALLDAYS_SUM ca ON c.Id = ca.Id;
	


END;
GO

### **<span style='color:cadetblue'>VIEWS</span>**

In [ ]:
-- 1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_VACCINATION_SERIES_NL] AS
   
       -- Add series attributes like VACCINE_SERIES_NL to Data
     WITH SERIES_CTE AS (
         SELECT
             A.DATE_OF_STATISTICS,
             r.[VACCINATION_KEY] AS [VACCINATION_KEY],
             r.[VACCINE_SERIES_ORDER] AS [VACCINE_CAMPAIGN_ORDER],
             r.[VACCINE_SERIES_NL] AS [VACCINE_CAMPAIGN_NAME_NL],
             A.[VACCINE_SERIE_EN] AS [VACCINE_CAMPAIGN_NAME_EN],
             A.[CUMSUM_TOTAL_VACCINATED] AS [VACCINE_ADMINISTERED_TOTAL],
             A.[TOTAL_VACCINATED_7DAYS] AS [VACCINE_ADMINISTERED_LAST_WEEK],
             [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_START_ISO](A.[DATE_OF_STATISTICS])) AS [DATE_UNIX],
             [dbo].[CONVERT_DATETIME_TO_UNIX]([dbo].[WEEK_START_ISO](A.[DATE_OF_STATISTICS])) AS [DATE_END_UNIX],
             [dbo].[CONVERT_DATETIME_TO_UNIX](dateadd(day, -6,  [dbo].[WEEK_START_ISO](A.[DATE_OF_STATISTICS]))) AS [DATE_START_UNIX],
             [dbo].[CONVERT_DATETIME_TO_UNIX](A.[DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
         FROM [VWSDEST].[RIVM_VACCINATION_SERIES_NL] A
         LEFT JOIN (
             SELECT
                 VACCINATION_KEY AS VACCINATION_KEY,
                 LABEL_EN AS VACCINE_SERIES_EN,
                 LABEL_NL AS VACCINE_SERIES_NL,
                 SORT_ORDER AS [VACCINE_SERIES_ORDER],
                 SERIES_START_DATE AS [VACCINE_SERIES_START_DATE],
                 SERIES_END_DATE AS [VACCINE_SERIES_END_DATE]
             FROM [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA] 
             WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA]) 
         ) r ON A.VACCINE_SERIE_EN = r.VACCINATION_KEY
         WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[RIVM_VACCINATION_SERIES_NL] ) 
     ),
     -- Construct Total at Start_Date
     TOTAL_START_DATE_202209 AS (
         SELECT DATE_OF_INSERTION_UNIX, SUM([VACCINE_ADMINISTERED_TOTAL]) as [VACCINE_ADMINISTERED_TOTAL_START_DATE]
         FROM SERIES_CTE A
         WHERE NOT [VACCINE_CAMPAIGN_NAME_EN] IN ('primary_completed','primary_started')
             AND DATE_OF_STATISTICS = DATEADD(dd, -1,            
                 (SELECT SERIES_START_DATE AS VACCINE_SERIES_START_DATE
                 FROM [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA] 
                 WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA]) 
                     AND LOWER(VACCINATION_KEY) = '202209_campaign'))
         GROUP BY A.DATE_OF_INSERTION_UNIX
     ),
     -- Construct Campaign 202209
     CAMPAIGN_202209_SERIES AS (
         SELECT
             A.DATE_OF_STATISTICS,
             r.[VACCINATION_KEY] AS [VACCINATION_KEY],
             r.[VACCINE_SERIES_ORDER] AS [VACCINE_CAMPAIGN_ORDER],
             r.[VACCINE_SERIES_NL] AS [VACCINE_CAMPAIGN_NAME_NL],
             r.[VACCINE_SERIES_EN] AS [VACCINE_CAMPAIGN_NAME_EN],
             SUM([VACCINE_ADMINISTERED_TOTAL]) - MAX([VACCINE_ADMINISTERED_TOTAL_START_DATE]) as [VACCINE_ADMINISTERED_TOTAL],
             SUM([VACCINE_ADMINISTERED_LAST_WEEK]) as [VACCINE_ADMINISTERED_LAST_WEEK],
             [DATE_UNIX],
             [DATE_START_UNIX],
             [DATE_END_UNIX],
             A.[DATE_OF_INSERTION_UNIX]
         FROM SERIES_CTE A
         -- Get Vaccine_Series name and Order
         CROSS JOIN (
             SELECT
                 LABEL_EN AS [VACCINE_SERIES_EN],
                 LABEL_NL AS [VACCINE_SERIES_NL],
                 SORT_ORDER AS [VACCINE_SERIES_ORDER]
             FROM [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA] 
             WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA]) 
                 AND LOWER(VACCINATION_KEY) = '202209_campaign' 
         ) r 
         -- Get Total per Start_Date
         CROSS JOIN TOTAL_START_DATE_202209 B
         -- Summarize for everything except for the two named series
         WHERE NOT LOWER(A.[VACCINE_CAMPAIGN_NAME_EN]) IN ('primary_completed','primary_started')
         GROUP BY 
             A.DATE_OF_STATISTICS,
             r.[VACCINE_SERIES_ORDER],
             r.[VACCINE_SERIES_NL],
             r.[VACCINE_SERIES_EN],
             [DATE_UNIX],
             [DATE_START_UNIX],
             [DATE_END_UNIX],
             A.[DATE_OF_INSERTION_UNIX]
     ),
     MaxDate As (
         SELECT MAX([DATE_OF_STATISTICS]) As MaxDate_Of_Statistics
         FROM SERIES_CTE 
         WHERE (LOWER(VACCINE_CAMPAIGN_NAME_EN) = 'primary_completed')
     )
     SELECT DATE_OF_STATISTICS, 
             VACCINE_CAMPAIGN_ORDER,
             VACCINE_CAMPAIGN_NAME_NL,
             VACCINE_CAMPAIGN_NAME_EN,
             VACCINE_ADMINISTERED_TOTAL,
             VACCINE_ADMINISTERED_LAST_WEEK,
             DATE_UNIX,
             DATE_END_UNIX,
             DATE_START_UNIX,
             DATE_OF_INSERTION_UNIX
     FROM SERIES_CTE 
        CROSS JOIN MaxDate
     WHERE [DATE_OF_STATISTICS] = MaxDate.MaxDate_Of_Statistics
         AND (LOWER(VACCINE_CAMPAIGN_NAME_EN) = 'primary_completed')
 
     UNION
     
     SELECT  DATE_OF_STATISTICS,
             VACCINE_CAMPAIGN_ORDER,
             VACCINE_CAMPAIGN_NAME_NL,
             VACCINE_CAMPAIGN_NAME_EN,
             VACCINE_ADMINISTERED_TOTAL,
             VACCINE_ADMINISTERED_LAST_WEEK,
             DATE_UNIX,
             DATE_END_UNIX,
             DATE_START_UNIX,
             DATE_OF_INSERTION_UNIX
     FROM CAMPAIGN_202209_SERIES
     WHERE [DATE_OF_STATISTICS] = (SELECT MAX([DATE_OF_STATISTICS]) FROM  CAMPAIGN_202209_SERIES)
 
GO

### **<span style='color:cadetblue'>VIEWS | CONFIGURATION</span>**

In [ ]:
-- View aanmaken in DataTino_Protos_1.Views
DECLARE @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINATION_SERIES_NL%')
DELETE FROM DATATINO_PROTO_1.CONFIGURATIONS WHERE View_ID = @ViewID
DELETE FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINATION_SERIES_NL%'

INSERT INTO Datatino_Proto_1.Views (Last_Update_name, Constraint_Key_name, Constraint_Value, Grouped_Key_Name, Grouped_Last_Update_Name, Name, Description)
SELECT 'DATE_UNIX',
       NULL,
       NULL,
       NULL,
       NULL,
       'VWSDEST.V_RIVM_VACCINATION_SERIES_NL',
       'VIEW: VWSDEST.V_RIVM_VACCINATION_SERIES_NL FOR vaccine_campaigns/vaccine_campaigns'
       

In [ ]:
-- Naam van bestaande configuratie aanpassen naar _disabled en Active naar 0
DECLARE  @ViewID INT = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINE_CAMPAIGNS_AND_PLANNED%')
delete from  [DATATINO_PROTO_1].[CONFIGURATIONS] 
WHERE VIEW_ID = @ViewID
delete from  [DATATINO_PROTO_1].[VIEWS] WHERE ID = @ViewID


-- Twee regels toevoegen aan Datatino_Proto_1.Configurations
SET @ViewID = (SELECT ID FROM Datatino_Proto_1.Views WHERE Name Like '%VWSDEST.V_RIVM_VACCINATION_SERIES_NL%')
DECLARE @ProtoID INT = (SELECT ID FROM Datatino_Proto_1.Protos WHERE Name = 'NL')

-- Eerst evt al aanwezige regels verwijderen
DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE Proto_ID = @ProtoID and View_ID = @ViewID

-- Regel toevoegen voor 'header'
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0, 
       'DATE_UNIX|DATE_END_UNIX|DATE_START_UNIX|DATE_OF_INSERTION_UNIX',
       '=LOWER()',
       3,
       NULL,
       1,
       'vaccine_campaigns',
       'VIEW CONFIGURATION: VWSDEST.V_RIVM_VACCINATION_SERIES_NL FOR vaccine_campaigns'

-- Regel toevoegen voor 'details'
INSERT INTO [DATATINO_PROTO_1].[CONFIGURATIONS] (Proto_ID, View_ID, Constrained, Grouped, Columns, Mapping, Layout_type_id, Mock_id, Active, Name, Description)
SELECT @ProtoID, 
       @ViewID, 
       0,
       0,
       'VACCINE_CAMPAIGN_ORDER|VACCINE_CAMPAIGN_NAME_NL|VACCINE_CAMPAIGN_NAME_EN|VACCINE_ADMINISTERED_TOTAL|VACCINE_ADMINISTERED_LAST_WEEK',
       '=LOWER()',
       4,
       NULL,
       1,
       'vaccine_campaigns/vaccine_campaigns',
       'VIEW CONFIGURATION: VWSDEST.V_RIVM_VACCINATION_SERIES_NL FOR vaccine_campaigns/vaccine_campaigns'
       